<a href="https://colab.research.google.com/github/thad75/OptionAI/blob/main/Project/Hands_On_Your_First_ML_Stack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Disclaimer : You will be accessing lots of free frameworks using your Google Account. Feel free to revoke them after the end of the Option AI Classes.

# Hands-On : Building your first simple stack. (3h)

<img src="https://cdn-lfs.huggingface.co/repos/96/a2/96a2c8468c1546e660ac2609e49404b8588fcf5a748761fa72c154b2836b4c83/9cf16f4f32604eaf76dabbdf47701eea5a768ebcc7296acc1d1758181f71db73?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27hf-logo.png%3B+filename%3D%22hf-logo.png%22%3B&response-content-type=image%2Fpng&Expires=1710601193&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMDYwMTE5M319LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85Ni9hMi85NmEyYzg0NjhjMTU0NmU2NjBhYzI2MDllNDk0MDRiODU4OGZjZjVhNzQ4NzYxZmE3MmMxNTRiMjgzNmI0YzgzLzljZjE2ZjRmMzI2MDRlYWY3NmRhYmJkZjQ3NzAxZWVhNWE3NjhlYmNjNzI5NmFjYzFkMTc1ODE4MWY3MWRiNzM%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qJnJlc3BvbnNlLWNvbnRlbnQtdHlwZT0qIn1dfQ__&Signature=TczHubMkO7sMQxJKNGcmSmf-sZS%7E83h7X6duKusZrR8cra12wUABImnMsH8YM1tVSAZll76Pv99baOONGMH4RhaLBAlwIyzWX8erIyamju1Jt0Gf%7EP1dQzJ6UpxR-yjHdIPiHM2DSBgHUnUsgBnO11Xrk2e0qN1wMvApi4vnx0yaiefGMal46AWmn8-oxA5QecoI3-PobGX-IFunsTbecKo9WCfgsezeA4HwGXENhlG%7E79feGAz3KO2TdJBm0pOSJnB4Gxu5RIERj1%7E4w1-RBnEA12xDpvYwNztyWrw05gfk6dZ3FwEqVtd97GmTJtXNF38fYmfINJwYK8wfa82JtA__&Key-Pair-Id=KVTP0A1DKRTAX"  width="200" height="200">

<img src='https://pypi-camo.freetls.fastly.net/a95ef5913dc4cc84d2155ff690a0fa0d4c33d7e2/68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f67726164696f2d6170702f67726164696f2f6d61696e2f726561646d655f66696c65732f67726164696f2e737667'  width="200" height="200">

<img src = "https://raw.githubusercontent.com/wandb/assets/main/wandb-logo-yellow-dots-black-wb.svg"  width="200" height="200">


Welcome to the world of AI engineering! In this course, we're going to build a stack for training a summarization model. Don't worry if you don't grasp every minute detail of the frameworks introduced – our main objectives are to:

1. Get acquainted with future MLOps practices.
2. Engage in hands-on AI practice, constructing a partially functional stack.
3. Create something meaningful that you can proudly include on your resume.
4. Open the door to internship opportunities.

Let's embark on this journey together and lay the foundation for your AI engineering expertise.

**You will encounter lots of issues in this lab. And it is completely normal. Feel free to think A LOT before asking any questions.**

In [ ]:
!pip install -q gradio
!pip install wandb -qU
!pip install transformers datasets evaluate rouge_score
!pip install accelerate -U

# I - Data Layer

## Load and Explore


As always, data forms the foundation of our ML systems. The principle of GIGO (Garbage in, garbage out) emphasizes that the quality of input data significantly impacts the performance of your production system.

We have a basic dataset for subsequent parts, starting with the data layer. Let's delve into the cnn_dailymail dataset, exploring and analyzing its contents.

In [ ]:
from datasets import load_dataset

dataset = load_dataset("cnn_dailymail", '3.0.0', split=['train',
                                                        'validation',
                                                        'test[:1%]'])


In [ ]:
# TODO : What format is dataset ?
# TODO : Does it contain everything we need for a good training ?
dataset

In [ ]:
# TODO : Pick the sample at index 0 from the dataset train. What keys are present ? What do they characterize ?
dataset...

In [ ]:
# TODO : Pick the sample at index 0 from the dataset valid. What keys are present ? What do they characterize ?
dataset...

In [ ]:
# TODO : Pick the sample at index 0 from the dataset test. What keys are present ? What do they characterize ?
dataset...

# II - Data to Model

Now, let's talk about the key tools we need: a **model** and its sidekick. In the NLP world, the trusty sidekick is called a **tokenizer**, and it's pretty much the model's best friend.
We will leverage from the Seq2Seq model.

### a - Tokenizer and Preprocessing

Humans understand natural language. However a model does not. As mentionned in the class, we represent words or subwords as tokens. Tokens are tensors that gave a representation within a space. Each pretrained model has its own representation space, based on the training vocab.
We are going to translate the whole dataset into tokens.
This process will take around 10 min.

In [ ]:
from transformers import AutoTokenizer

checkpoint = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Once the tokenizer loaded, we can either have to remap the training dataset to tokens. Obviously, there are functions that does that. In this case, we have to tokenize the input and also the labels.

In [ ]:
prefix = "summarize: "

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs,
                             max_length=1024,
                             truncation=True)
    labels = tokenizer(text_target=examples["highlights"],
                       max_length=128,
                       truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# TODO : Map your datasets using your tokenizers.
tokenized_train = dataset[...].map(preprocess_function, batched=True)
tokenized_valid = dataset[...].map(preprocess_function, batched=True)
tokenized_test = dataset[...].map(preprocess_function, batched=True)

Map:   0%|          | 0/134 [00:00<?, ? examples/s]

After preparing the dataset, you can set up a datacollator, which essentially works like a dataloader. It organizes the data into batches and can also apply specific random changes if needed. Depending on your project, you might opt to craft custom Dataloaders. In our situation, we'll stick with the pre-existing one provided by HuggingFace.

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer,
                                       model=checkpoint)

### b - Evaluation Metrics

To assess NLP models for summarization, ROUGE metrics are commonly used. ROUGE, which stands for Recall-Oriented Understudy for Gisting Evaluation, compares automatically generated summaries or translations to human-produced reference summaries or translations.

ROUGE metrics provide scores between 0 and 1, where a higher score indicates greater similarity between the automatically generated summary and the reference summary.

Rather than reinventing the wheel, let's leverage the existing metric computation suite. It's likely that another engineer has already done the work. Our approach will involve building upon the existing solutions rather than starting from scratch.

In [ ]:
import evaluate

# TODO : Load the rouge evaluation
rouge = evaluate.load(...)

To integrate the evaluation pipeline into our Model Layer, we'll need to customize it to suit our specific needs. This personalization ensures that the evaluation process aligns seamlessly with the requirements of our model and enhances its usability within the broader Model Layer.

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    return {k: round(v, 4) for k, v in result.items()}

### c - Loading Model

Let's efficiently load the model from the pre-trained checkpoint to capitalize on the knowledge acquired during pre-training. This step allows us to build upon the expertise encoded in the pre-existing model.

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

### d - Train the model you must

Becoming an AI engineer is where the fun begins! Models often respond differently to various hyperparameters. To explore and find the best configuration, we typically prepare multiple scripts with different hyperparameters. These configurations are stored in structured data formats like .YAML or .JSON files. HuggingFace provides a Seq2Seq training argument class that we can utilize to set up and customize our model with ease. This allows us to experiment with hyperparameters and optimize the model's performance.

Previously, for logging, we utilized Tensorboard, a free monitoring tool, but it had numerous bugs. Companies often turn to alternatives like W&B and MLFlow for model monitoring. In this lab, we'll be using W&B, a sleek dashboard. The cool part with HuggingFace is that it automatically manages the monitoring module by checking your environment.

In [ ]:
# TODO : Fill the arguments using your knowledge
training_args = Seq2SeqTrainingArguments(
    output_dir="....", # TODO : Give an output directory name for you project.
    evaluation_strategy="epoch",
    learning_rate= ...,    # TODO : Setup a learning rate. Remember that we are fine-tuning.
    per_device_train_batch_size=...,
    per_device_eval_batch_size=...,
    weight_decay=....,
    save_total_limit=...,
    num_train_epochs=...,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=False, # If you have a HF Account and want to push your trained model
)


It's time to create a Trainer – the class that will take charge of the training process for us. This class streamlines the training workflow, making the entire process more manageable and efficient.

The model should take 10 min to train. If you have any questions feel free to ask them

In [ ]:
# TODO : Initialize the model with the correct arguments
trainer = Seq2SeqTrainer(
    model= ...,
    args= ...,
    train_dataset= ...,
    eval_dataset= ...,
    tokenizer= ...,
    data_collator= data_collator,
    compute_metrics= compute_metrics,
)

# Train
trainer.train()

# III - Serving


Now that we can assess that our model works, we have to deploy it to the end user. Typically, someone that is going to use the model but on an interface like what ChatGPT,Bard does. Deploiement could also be done on hardware materials. But we won't go into that field.

In this way, we will be leveraging from Gradio. Gradio serves as a simple tool to quickly develop a interface for serving. However, in reality there's lot more going behind. In companies, we create API with separate back and front ends to fullfil the need of the client.

Let's plug our model to gradio and test the front end interface given to us.

In [ ]:
from gradio import Interface

name = '...'

def summarize(text, sumup = True):
    # TODO : Fill the missing stuff from the text pipeline
    input_ids = tokenizer(...,
                          return_tensors="pt")["input_ids"]
    output = model.generate(input_ids.to(model.device))
    summary = tokenizer.decode(output[0],
                               skip_special_tokens=True)
    return summary

# Create the Gradio interface
interface = Interface(
    fn=summarize,
    inputs="text",
    outputs="text",
    title=f"{name}'s first ML Stack Deployed",
    description="Enter text to get a summary using your seq2seq model."
)

# Launch the Gradio interface
interface.launch(debug = True,
                 share = True)

If you notice problems with your model's predictions, there's a reason for it. We encourage you to investigate the cause behind these issues. Now, let's get back to the class.